# Пример получения информации с помощью moex_bond_info

In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True, precision=2)
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 100

from jinja2 import Environment, FileSystemLoader
from moex_bonds import isin_secid, moex_bond_info, moex_bond_yield

Различные типы облигаций для тестирования:
|Тип облигации|ISIN|
|---|---|
|Корпоративная (plain vanilla)|RU000A105U00|
|Корпоративная (floater)|RU000A107HR8|
|Корпоративная (с амортизацией долга)|RU000A106JZ9|
|Корпоративная (call option)|RU000A100X69|
|Корпоративная (put option)|RU000A101QL5|
|Корпоративная (с амортизацией долга + call option)|RU000A100T81|
|Государственная (ОФЗ-ПД)|RU000A0JS3W6|
|Государственная (ОФЗ-ПК)|RU000A0JV4P3|

\* - для ОФЗ-ПК действует правило: при неизвестных значениях купонов, эти значения берутся равными последней известной купонной ставке, если иное не определено в эмиссионных документах.

Вспомагательная функция *isin_secid* для конвертации ISIN в SECID и обратно.

|Direction|From code|To code|
|---|---|---|
|isin2secid|RU000A0JS3W6|SU26207RMFS9|
|secid2isin|SBER|RU0009029540|


Получение данных по облигации:

In [2]:
df_nfo, df_cf = moex_bond_info(isin_secid('RU000A105U00'))

In [3]:
df_nfo

,key,value
SECID,Код ценной бумаги,RU000A105U00
ISIN,ISIN код,RU000A105U00
NAME,Полное наименование,Газпром капитал ООО БО-001Р-08
SHORTNAME,Краткое наименование,ГазпромКP8
LISTLEVEL,Уровень листинга,2
ISQUALIFIEDINVESTORS,Для квал. инвесторов,0
ISSUESIZE,Объем выпуска,30.0 млрд
INITIALFACEVALUE,Первоначальная номн. стоимость,1000
FACEUNIT,Валюта номинала,SUR
DAYSTOREDEMPTION,Дней до погашения,514


In [4]:
df_cf

,event_date,coupon,amt,offer,offer_type
ГазпромКP8,,,,,
1,2023-08-11,45.87,–,–,–
2,2024-02-09,45.87,–,–,–
3,2024-08-09,45.87,–,–,–
4,2025-02-07,45.87,–,–,–
5,2025-08-08,45.87,–,–,–
6,2026-02-06,45.87,"1,000.00",–,–


Дальнейшее использование полученной информации:
* Расчет доходности (для простых облигаций на текущую дату)

In [5]:
price_lo = np.floor(df_nfo.loc['PREVWAPRICE', 'value'] - 1)
price_hi = np.ceil(df_nfo.loc['PREVWAPRICE', 'value'] + 1) + 0.1

for price in np.arange(price_lo, price_hi, 0.2):
    print(f'Price: {price:<6.2f} YTM: {moex_bond_yield(df_nfo, df_cf, price):.2f}')

Price: 87.00  YTM: 21.25
Price: 87.20  YTM: 21.05
Price: 87.40  YTM: 20.84
Price: 87.60  YTM: 20.64
Price: 87.80  YTM: 20.43
Price: 88.00  YTM: 20.23
Price: 88.20  YTM: 20.03
Price: 88.40  YTM: 19.83
Price: 88.60  YTM: 19.63
Price: 88.80  YTM: 19.43
Price: 89.00  YTM: 19.23
Price: 89.20  YTM: 19.03
Price: 89.40  YTM: 18.83
Price: 89.60  YTM: 18.64
Price: 89.80  YTM: 18.44
Price: 90.00  YTM: 18.25


* Генерация HTML отчета

In [6]:
loader = FileSystemLoader('bond_reports/template')
environment = Environment(loader=loader)
template = environment.get_template('template.html')
result = template.render({'nfo': df_nfo, 'cf': df_cf})

with open(
    f'bond_reports/{df_nfo.loc["SHORTNAME", "value"]}, {df_nfo.loc["ISIN", "value"]}.html', 
    mode='w', 
    encoding='utf-8'
) as fn:
        fn.write(result)